### Install Libraries

In [1]:
# Used to open and process GIF images
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


### Import Libraries

In [1]:
'''This imports the NumPy library and aliases it as "np." NumPy is a fundamental package for numerical computing in Python, 
and it provides support for arrays, matrices, and mathematical functions.'''
import numpy as np

'''This imports the TensorFlow library and aliases it as "tf." 
TensorFlow is an open-source machine learning framework developed by Google for various machine learning and deep learning tasks.'''
import tensorflow as tf

'''his imports the "Sequential" class from the "tensorflow.keras.models" module. 
In TensorFlow, the "Sequential" class is commonly used to create sequential neural network models, where you stack layers sequentially.'''
from keras.models import Sequential

'''This imports the "Dense" and "Flatten" layer classes from the "tensorflow.keras.layers" module. These layers are building blocks 
for creating neural network architectures."Dense" represents a fully connected layer, and "Flatten" is used to flatten the input data.'''
from keras.layers import Dense, Flatten

'''This imports the Stochastic Gradient Descent (SGD) optimizer from the "tensorflow.keras.optimizers" module. 
SGD is a popular optimization algorithm used for training neural networks.'''
from keras.optimizers import SGD

'''This imports the "GridSearchCV" class from the "sklearn.model_selection" module. 
It's part of the Scikit-learn library and is used for hyperparameter tuning through grid search.'''
from sklearn.model_selection import GridSearchCV

''' This imports various evaluation metrics and the confusion matrix function from the "sklearn.metrics" module. 
These metrics are commonly used to assess the performance of machine learning models, including accuracy, precision, recall, and F1-score.'''
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

''' This imports the Matplotlib library and aliases it as "plt." 
Matplotlib is a popular library for creating visualizations, including plots and charts.'''
import matplotlib.pyplot as plt

'''To read the GIF images'''
from PIL import Image
import os

"""This is needed to split the data into train, test, and validation set"""
from sklearn.model_selection import train_test_split

from keras.wrappers.scikit_learn import KerasClassifier



### Task 1

#### Multi layer perceptron class

In [12]:
# Class to implement a multi layer perceptron for facial recognition
class MLP:
    
    '''Constructor to get important problem information:
    1) num_classes: The number of output classes
    2) image_width: The number of pixels in the horizontal direction
    3) image_height: The number of pixels in the vertical direction
    4) num_channels: The number of channels in the image'''
    def __init__(self, num_classes, image_width, image_height, num_channels):
        self.num_classes = num_classes
        self.image_width = image_width
        self.image_height = image_height
        self.num_channels = num_channels
    

    '''Function to create the model. It has the following parameters
    1) neurons_per_layer: The number of neurons (units) in each hidden layer of the neural network. The default is set to 64.
    2) num_hidden_layer: The number of hidden layers in the neural network. The default is set to 2.
    3) activation:  The activation function used in the hidden layers. The default is set to 'relu' (Rectified Linear Unit).
    4) learning_rate: : The learning rate for the stochastic gradient descent (SGD) optimizer, which controls the step size during training. The default is set to 0.01.
    5) momentum: The momentum term for SGD, which helps accelerate training. The default is set to 0.9.'''
    def create_model(self, neurons_per_layer=64, num_hidden_layers=2, activation='relu', learning_rate=0.01, momentum=0.9):
        # Create an instance of a sequential neural network model of the TensorFlow's Sequential class which allows to build neural network by stacking layers sequentially 
        model = Sequential()

        # Add a flattened input layer used to flatter the input data. The input shape parameter specifies the shape of the input data(image width,height, and number of channels)
        model.add(Flatten(input_shape=(self.image_width, self.image_height, self.num_channels)))

        # Loop to add multiple dense(fully connected) hidden layers to the network
        for _ in range(num_hidden_layers):
            model.add(Dense(neurons_per_layer, activation=activation))

        # Add an output dense layer having neurons equal to number of classes. The activation function is 'softmax,' which is common for multiclass classification problems.
        model.add(Dense(self.num_classes, activation='softmax'))

        # An instance of the stochastic gradient descent (SGD) optimizer is created with the specified learning_rate and momentum parameters.
        optimizer = SGD(learning_rate=learning_rate, momentum=momentum)

        # Compile the model specifying the optimizer, loss function('categorical_crossentropy' for multiclass classification) and the metrics to track during training e.g accuracy
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # return the compiler neural network model
        return model

    '''Function to train the model. It has the following parameters:
    1) X_train: The input to the training data
    2) Y_train: The output to the training data
    3) epochs: Specifies the number of training epochs, which is the number of times the model will be trained on the entire training dataset.
    4) batch_size: Indicates how many samples from the training dataset are used in each iteration of training'''
    def fit(self, X_train, y_train, epochs=10, batch_size=32):

        # Create an instance of a neural network model
        model = self.create_model()

        # Start the training process. Verbose = 1 means the training progress will be showed
        history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

        # Return the trained model and its history(information)
        return model, history

    ''' Function to test the model's perfomance on unseen data. It has the following parameters:
    1) model: The model to test
    2) X_test: The input to the test data
    3) y_test: The output to the test data'''
    def test(self, model, X_test, y_test):

        # Use the trained model to make predictions on the test data
        y_pred = model.predict(X_test)

        # 
        y_pred = np.argmax(y_pred, axis=1)
        accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
        return accuracy

    '''Function to tune the model
    def tune(self, X_validation, y_validation, param_grid, cv=3):

        model = self.create_model()

        # Create a KerasClassifier for grid search
        keras_classifier = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=lambda: model, verbose=1)

        # Initialize GridSearchCV
        grid = GridSearchCV(estimator=keras_classifier, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=1)

        # Reshape y_train to work with GridSearchCV
        y_validation = np.array(y_validation)

        # Perform grid search
        grid_result = grid.fit(X_validation, y_validation)

        # Get the best model and its hyperparameters
        best_params = grid_result.best_params_

        return best_params
    '''
    def tune(self, X_train, y_train, param_grid, cv=2):

        model = KerasClassifier(build_fn=self.create_model, epochs=10, verbose=1)

        grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=cv)

        grid_result = grid.fit(X_train, y_train)

        return grid_result



#### Read the Data

In [3]:
# Function to read GIF images from a directory
def read_gif_images(directory):

    # Store the image data
    image_data = []

    # Store image name
    file_names = []

    # Iterate through the files
    for file in os.listdir(directory):
    
        # If the file is a GIF image, the code constructs the full file path by joining the specified directory (directory) and the current file name (file) using os.path.join
        file_path = os.path.join(directory, file)

        # Append the filename to the file_name list
        file_names.append(file)

        # Open the GIF image
        image = Image.open(file_path)

        # Convert the images to grayscale(They are already black and white)
        image = image.convert('RGB')

        # Append the opened image to image_data list
        image_data.append(image)

    # Return the GIF images, and there file names
    return image_data, file_names

In [4]:
# Read the data
image_data, file_names = read_gif_images("Data")

#### Display the Data

In [ ]:
import matplotlib.pyplot as plt

# Assuming image_data contains PIL Image objects
def display_images(image_data, file_names):
    num_images = len(image_data)

    # Determine the number of rows and columns for subplots
    rows = int(num_images / 4)  # Adjust the number of columns as needed
    if num_images % 4 != 0:
        rows += 1

    # Create subplots for displaying images
    fig, axes = plt.subplots(rows, 4, figsize=(12, 3 * rows))
    fig.subplots_adjust(hspace=0.5)
    
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            ax.imshow(image_data[i])
            ax.set_title(file_names[i])
            ax.axis('off')
        else:
            ax.axis('off')

    plt.show()

# Call the display_images function
display_images(image_data, file_names)


#### Split the Data

In [5]:
# Determine split ratios
train_ratio = 0.7
test_ratio = 0.3

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    image_data, file_names, test_size=test_ratio, random_state=42, shuffle=True
)

# Extract only the subject information from the file names
y_train = [file_name.split('.')[0] for file_name in y_train]
y_test = [file_name.split('.')[0] for file_name in y_test]

# Convert PIL Image objects to NumPy arrays
X_train = [np.array(image) for image in X_train]
X_test = [np.array(image) for image in X_test]

X_train = np.array(X_train)
X_test = np.array(X_test)

# Convert labels to numerical values (example: integer encoding)
label_mapping = {label: index for index, label in enumerate(set(y_train))}
y_train = [label_mapping[label] for label in y_train]
y_test = [label_mapping[label] for label in y_test]

y_train = np.array(y_train)
y_test = np.array(y_test)

print(f"Number of samples in training set: {len(X_train)}")
print(f"Number of samples in test set: {len(X_test)}")


Number of samples in training set: 115
Number of samples in test set: 50


In [9]:
np.shape(X_train)

(115, 243, 320, 3)

#### Start the training

In [13]:
# Dictionary that defines a grid of hyperparameter values to search through in grid seach process

'''
param_grid = {
    'neurons_per_layer': [32, 64, 128],
    'num_hidden_layers': [1, 2, 3],
    'activation': ['relu', 'sigmoid'],
    'epochs': [10, 20, 30],
    'learning_rate': [0.001, 0.01, 0.1],
    'momentum': [0.5, 0.9, 0.99]
}
'''
param_grid = {
    'neurons_per_layer': [12, 22, 32],
    'num_hidden_layers': [5, 10, 15]
}

# Initialize MLP object
mlp = MLP(num_classes=15, image_width=243, image_height=320, num_channels=3)

# Perform hyperparameter tuning
best_params = mlp.tune(X_train=X_train, y_train=y_train, param_grid= param_grid)


C:\Users\ubaid\AppData\Local\Temp\ipykernel_15056\3508181880.py:99: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=self.create_model, epochs=10, verbose=1)


Epoch 1/10
2/2 [==============================] - 1s 32ms/step - loss: 375348.7500 - accuracy: 0.0175
Epoch 2/10
2/2 [==============================] - 0s 36ms/step - loss: 219450441728.0000 - accuracy: 0.1053
Epoch 3/10
2/2 [==============================] - 0s 37ms/step - loss: 713189752832.0000 - accuracy: 0.0877
Epoch 4/10
2/2 [==============================] - 0s 38ms/step - loss: 20121.8809 - accuracy: 0.0702
Epoch 5/10
2/2 [==============================] - 0s 53ms/step - loss: 2.7118 - accuracy: 0.0877
Epoch 6/10
2/2 [==============================] - 0s 33ms/step - loss: 2.7125 - accuracy: 0.0877
Epoch 7/10
2/2 [==============================] - 0s 32ms/step - loss: 2.7126 - accuracy: 0.0877
Epoch 8/10
2/2 [==============================] - 0s 37ms/step - loss: 2.7124 - accuracy: 0.0877
Epoch 9/10
2/2 [==============================] - 0s 32ms/step - loss: 2.7121 - accuracy: 0.0877
Epoch 10/10
2/2 [==============================] - 0s 11ms/step - loss: 2.7015 - accuracy: 0.069

In [16]:
best_params.best_params_

{'neurons_per_layer': 22, 'num_hidden_layers': 5}

In [ ]:
# Train the best model
trained_model, history = mlp.fit(X_train, y_train, epochs=best_params['epochs'])

# Test the model
test_accuracy = mlp.test(trained_model, X_test, y_test)

print(f"Best Hyperparameters: {best_params}")
print(f"Test Accuracy with Best Model: {test_accuracy}")

In [ ]:


''' Create an instacne of keras classifier which allows keras based model with scikit learn.
The build_fn is set to the create_model function which is used to create different model instances during
the grid search with various hyperparameters. Verbose is set to 0(training progess wont be displayed)
'''
model = tf.keras.wrappers.scikit_learn.KerasClassifier(build_fn=create_model, verbose=0)

'''Create an instance of the GridSearchCV class which performs a grid search over the specified hyperparamter grid
It has the following parameters
1) estimator: The model used in this case, The keras classifier model defined
2) param_grid: The hyperparameters used in the search
3) cv: Number of fold cross validation used for evaluating the models performance
4) scoring: The metric used to evaluate the model
5) n_jobs: -1 means that it will use all CPU cores for parallel computaion'''
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=1)

# Start the Grid search by fitting the grid search object to the training data
grid_result = grid.fit(X_train, y_train)


In [ ]:
y_pred = grid_result.best_estimator_.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


### Task 4